## DESI Y3 redshift 'ERRORS' Analysis

This notebook analyzes redshift 'ERRORS' (uncertainties and catastrophics) in DESI Y3 Abacus mock data 

In [2]:
import os
import sys
import glob
import fitsio
import numpy as np
from astropy.io import fits
from astropy.table import Table, vstack
from matplotlib import pyplot as plt

from cosmoprimo.fiducial import AbacusSummit

plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['legend.fontsize'] = 14
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

sys.path.append('/global/homes/s/shengyu/project_rc/main/Y3/')
from helper import REDSHIFT_VSMEAR, Y3_NRAN, Y3_ZRANGE

In [3]:
c = 299792 # speed of light in km/s
cosmo = AbacusSummit() 

def get_edges(corr_type='smu', bin_type='lin'):
    if bin_type == 'log':
        sedges = np.geomspace(0.01, 100., 49)
    elif bin_type == 'lin':
        sedges = np.linspace(0., 200, 201)
    else:
        raise ValueError('bin_type must be one of ["log", "lin"]')
    if corr_type == 'smu':
        edges = (sedges, np.linspace(-1., 1., 201)) #s is input edges and mu evenly spaced between -1 and 1
    elif corr_type == 'rppi':
        if bin_type == 'lin':
            edges = (sedges, np.linspace(-40., 40, 101)) #transverse and radial separations are coded to be the same here
        else:
            edges = (sedges, np.linspace(-40., 40., 81))
    elif corr_type == 'theta':
        edges = (np.linspace(0., 4., 101),)
    else:
        raise ValueError('corr_type must be one of ["smu", "rppi", "theta"]')
    return edges

def _concatenate(arrays):
    if isinstance(arrays[0], (tuple, list)):  # e.g., list of bitwise weights for first catalog
        array = [np.concatenate([arr[iarr] for arr in arrays], axis=0) for iarr in range(len(arrays[0]))]
    else:
        array = np.concatenate(arrays, axis=0)  # e.g. individual weights for first catalog
    return array

def get_positions_weights(catalog, tracer, region, zrange, weight_type = 'default', systematics_type = None):
    z2d = cosmo.comoving_radial_distance
    toret = []
    if isinstance(catalog, (tuple, list)):  # list of catalogs, one for each region
        for cat in catalog:
            toret += get_positions_weights(cat, tracer, region, zrange, weight_type = 'default', systematics_type = None)
    else:
        if systematics_type == 'dv-obs':
            import time
            from Y3_redshift_systematics import vsmear, vsmear_modelling
            random_seed = int(time.time() * 1000) % (2**32)
            zmid = (zrange[0]+zrange[1])/2
            dv = vsmear(tracer, zmin=zrange[0], zmax=zrange[1], Ngal = len(catalog), seed=random_seed)
            dz = dv*(1+zmid)/c # ∆z = ∆v*(1+z)/c
            catalog['Z'] += dz
        maskz = (catalog['Z'] >= zrange[0]) & (catalog['Z'] < zrange[1]) 
        mask  = maskz & select_region(catalog['RA'], catalog['DEC'], region=region)
        cat = catalog[mask]
        dist = z2d(cat['Z'])
        positions = [cat['RA'].data, cat['DEC'].data, dist]
        weights = np.ones(len(cat), dtype=bool)
        if 'default' in weight_type:
            weights = cat['WEIGHT'].data
        toret.append((np.array(positions), np.array(weights)))
        # toret.append((positions, weights))
    return toret

def normalize_data_randoms_weights(data_weights, randoms_weights, weight_attrs=None):
    # Renormalize randoms / data for each input catalogs
    # data_weights should be a list (for each N/S catalogs) of weights
    import inspect
    from pycorr.twopoint_counter import _format_weights, get_inverse_probability_weight
    if weight_attrs is None: weight_attrs = {}
    weight_attrs = {k: v for k, v in weight_attrs.items() if k in inspect.getargspec(get_inverse_probability_weight).args}
    wsums, weights = {}, {}
    for name, catalog_weights in zip(['data', 'randoms'], [data_weights, randoms_weights]):
        wsums[name], weights[name] = [], []
        for w in catalog_weights:
            w, nbits = _format_weights(w, copy=True)  # this will sort bitwise weights first, then single individual weight
            iip = get_inverse_probability_weight(w[:nbits], **weight_attrs) if nbits else 1.
            iip = iip * w[nbits]
            wsums[name].append(iip.sum())
            weights[name].append(w)
    wsum_data, wsum_randoms = sum(wsums['data']), sum(wsums['randoms'])
    for icat, w in enumerate(weights['randoms']):
        factor = wsums['data'][icat] / wsums['randoms'][icat] * wsum_randoms / wsum_data
        w[-1] *= factor
        print('Rescaling randoms weights of catalog {:d} by {:.4f}.'.format(icat, factor), flush=True)
    return weights['data'], weights['randoms']

def concatenate_data_randoms(data, randoms=None, **kwargs):
    if randoms is None:
        positions, weights = data
        return _concatenate(positions), _concatenate(weights)
    positions, weights = {}, {}
    for name in ['data', 'randoms']:
        positions[name], weights[name] = locals()[name]
    for name in positions:
        concatenated = not isinstance(positions[name][0][0], (tuple, list))  # first catalog, unconcatenated [RA, DEC, distance] (False) or concatenated RA (True)?
        if concatenated:
            positions[name] = _concatenate(positions[name])
        else: 
            positions[name] = [_concatenate([p[i] for p in positions[name]]) for i in range(len(positions['randoms'][0]))]
    data_weights, randoms_weights = [], []
    if concatenated:
        wd, wr = normalize_data_randoms_weights(weights['data'], weights['randoms'], weight_attrs=kwargs.get('weight_attrs', None))
        weights['data'], weights['randoms'] = _concatenate(wd), _concatenate(wr)
    else:
        for i in range(len(weights['randoms'][0])):
            wd, wr = normalize_data_randoms_weights(weights['data'], [w[i] for w in weights['randoms']], weight_attrs=kwargs.get('weight_attrs', None))
            data_weights.append(_concatenate(wd))
            randoms_weights.append(_concatenate(wr))
        weights['data'] = data_weights[0]
        for wd in data_weights[1:]:
            for w0, w in zip(weights['data'], wd): assert np.all(w == w0)
        weights['randoms'] = randoms_weights
    return [(positions[name], weights[name]) for name in ['data', 'randoms']]


In [5]:
nran_list = {'ELG_LOPnotqso':10, 'LRG':8, 'QSO':4}

survey  ='DA2'
mockver ='Abacus_v4_1'
specver ='kibo-v1'
tracer  ='ELG_LOPnotqso' # LRG, ELG_LOPnotqso, QSO
nran = nran_list[tracer]
zrange = Y3_ZRANGE[tracer][0]
MOCKNUM = 0

file_path = f'/pscratch/sd/s/shengyu/galaxies/catalogs/{survey}/{mockver}/altmtl{MOCKNUM}/{specver}/mock{MOCKNUM}/LSScats'
catalog_fn = file_path+f'/{tracer}_clustering.dat.fits'
# catalog_fn = f'/global/cfs/cdirs/desi/survey/catalogs/DA2/mocks/SecondGenMocks/AbacusSummit_v4_1/altmtl0/kibo-v1/mock0/LSScats/QSO_clustering.dat.fits'
# print(catalog_fn)
# catalog=Table(fitsio.read(catalog_fn, columns=['RA', 'DEC', 'Z', 'WEIGHT']))
# print(catalog.colnames, '\n', len(catalog))

catalog_fn = file_path+f'/{tracer}_NGC_clustering.dat.fits'
print(catalog_fn)
catalog=Table(fitsio.read(catalog_fn, columns=['RA', 'DEC', 'Z', 'WEIGHT']))
print('Y3 Abacus data catalog:',len(catalog))

catalog_fn = file_path+f'/{tracer}_NGC_0_clustering.ran.fits'
print(catalog_fn)
catalog=Table(fitsio.read(catalog_fn, columns=['RA', 'DEC', 'Z', 'WEIGHT']))
print('Y3 Abacus random catalog:',len(catalog))

# catalog_fn = file_path+f'/{tracer}_SGC_clustering.dat.fits'
# print(catalog_fn)
# catalog=Table(fitsio.read(catalog_fn, columns=['RA', 'DEC', 'Z', 'WEIGHT']))
# print(catalog.colnames, '\n', len(catalog))

catalog_fn = f'/pscratch/sd/s/shengyu/galaxies/catalogs/Y1/Abacus_v4_2/altmtl0/iron/mock0/LSScats/{tracer}_NGC_clustering.dat.fits'
print(catalog_fn)
catalog=Table(fitsio.read(catalog_fn, columns=['RA', 'DEC', 'Z', 'WEIGHT']))
print('Y1 Abacus data catalog:',len(catalog))

catalog_fn = f'/pscratch/sd/s/shengyu/galaxies/catalogs/Y1/Abacus_v4_2/altmtl0/iron/mock0/LSScats/{tracer}_NGC_0_clustering.ran.fits'
print(catalog_fn)
catalog=Table(fitsio.read(catalog_fn, columns=['RA', 'DEC', 'Z', 'WEIGHT']))
print('Y1 Abacus random catalog:',len(catalog))

/pscratch/sd/s/shengyu/galaxies/catalogs/DA2/Abacus_v4_1/altmtl0/kibo-v1/mock0/LSScats/ELG_LOPnotqso_NGC_clustering.dat.fits
Y3 Abacus data catalog: 4279844
/pscratch/sd/s/shengyu/galaxies/catalogs/DA2/Abacus_v4_1/altmtl0/kibo-v1/mock0/LSScats/ELG_LOPnotqso_NGC_0_clustering.ran.fits
Y3 Abacus random catalog: 17296184
/pscratch/sd/s/shengyu/galaxies/catalogs/Y1/Abacus_v4_2/altmtl0/iron/mock0/LSScats/ELG_LOPnotqso_NGC_clustering.dat.fits
Y1 Abacus data catalog: 1813296
/pscratch/sd/s/shengyu/galaxies/catalogs/Y1/Abacus_v4_2/altmtl0/iron/mock0/LSScats/ELG_LOPnotqso_NGC_0_clustering.ran.fits
Y1 Abacus random catalog: 9825715


In [24]:
tracer = 'LRG'
region = 'NGC'
nsplits = 10
basedir = f'/pscratch/sd/s/shengyu/galaxies/catalogs/DA2/Abacus_v4_1/altmtl0/kibo-v1/mock0/LSScats'
ran_mock_fns = [basedir+f'/{tracer}_{region}_{{}}_clustering.ran.fits'.format(i) for i in range(2)]
ran_mocks = vstack([Table(fitsio.read(v, columns=['RA', 'DEC', 'Z', 'WEIGHT'])) for v in ran_mock_fns])
randoms = get_positions_weights(np.array_split(ran_mocks, nsplits), tracer, region, zrange, weight_type = 'default', systematics_type = None)

data_mock_fn = basedir+f'/{tracer}_NGC_clustering.dat.fits'
data_mocks=Table(fitsio.read(catalog_fn, columns=['RA', 'DEC', 'Z', 'WEIGHT']))
data = get_positions_weights(data_mocks, tracer, region, zrange, weight_type = 'default', systematics_type = None)

In [27]:
randoms_positions = [entry[0] for entry in randoms]
randoms_weights = [entry[1] for entry in randoms]
randoms = (randoms_positions, randoms_weights)

data_positions, data_weights = data
data = (data_positions, data_weights)

In [2]:
def get_edges(corr_type='smu', bin_type='lin'):
    if bin_type == 'log':
        sedges = np.geomspace(0.01, 100., 49)
    elif bin_type == 'lin':
        sedges = np.linspace(0., 200, 201)
    else:
        raise ValueError('bin_type must be one of ["log", "lin"]')
    if corr_type == 'smu':
        edges = (sedges, np.linspace(-1., 1., 201)) #s is input edges and mu evenly spaced between -1 and 1
    elif corr_type == 'rppi':
        if bin_type == 'lin':
            edges = (sedges, np.linspace(-40., 40, 101)) #transverse and radial separations are coded to be the same here
        else:
            edges = (sedges, np.linspace(-40., 40., 81))
    elif corr_type == 'theta':
        edges = (np.linspace(0., 4., 101),)
    else:
        raise ValueError('corr_type must be one of ["smu", "rppi", "theta"]')
    return edges

def get_positions_weights(catalog, tracer, region, zrange, weight_type = 'default', systematics_type = None):
    z2d = cosmo.comoving_radial_distance
    toret = []
    if isinstance(catalog, (tuple, list)):  # list of catalogs, one for each region
        for cat in catalog:
            toret += get_positions_weights(cat, tracer, region, zrange, weight_type = 'default', systematics_type = None)
    else:
        if systematics_type == 'dv-obs':
            import time
            from Y3_redshift_systematics import vsmear, vsmear_modelling
            random_seed = int(time.time() * 1000) % (2**32)
            zmid = (zrange[0]+zrange[1])/2
            dv = vsmear(tracer, zmin=zrange[0], zmax=zrange[1], Ngal = len(catalog), seed=random_seed)
            dz = dv*(1+zmid)/c # ∆z = ∆v*(1+z)/c
            catalog['Z'] += dz
        maskz = (catalog['Z'] >= zrange[0]) & (catalog['Z'] < zrange[1]) 
        mask  = maskz & select_region(catalog['RA'], catalog['DEC'], region=region)
        cat = catalog[mask]
        dist = z2d(cat['Z'])
        positions = [cat['RA'].data, cat['DEC'].data, dist]
        weights = np.ones(len(cat), dtype=bool)
        if 'default' in weight_type:
            weights = cat['WEIGHT'].data
        # toret.append((np.array(positions), np.array(weights)))
        toret.append((np.array(positions), np.array(weights)))
    return toret

def normalize_data_randoms_weights(data_weights, randoms_weights, weight_attrs=None):
    if mpicomm is None:
        mpicomm = mpi.COMM_WORLD
    # Renormalize randoms / data for each input catalogs
    # data_weights should be a list (for each N/S catalogs) of weights
    import inspect
    from pycorr.twopoint_counter import _format_weights, get_inverse_probability_weight
    if weight_attrs is None: weight_attrs = {}
    weight_attrs = {k: v for k, v in weight_attrs.items() if k in inspect.getargspec(get_inverse_probability_weight).args}
    def sum_weights(*weights):
        sum_weights, formatted_weights = [], []
        for weight in weights:
            weight, nbits = _format_weights(weight, copy=True)  # this will sort bitwise weights first, then single individual weight
            iip = (get_inverse_probability_weight(weight[:nbits], **weight_attrs) if nbits else 1.) * weight[nbits]
            sum_weights.append(mpicomm.allreduce(np.sum(iip)))
            formatted_weights.append(weight)
        return sum_weights, formatted_weights
    data_sum_weights, data_weights = sum_weights(*data_weights)
    randoms_sum_weights, randoms_weights = sum_weights(*randoms_weights)
    all_data_sum_weights, all_randoms_sum_weights = sum(data_sum_weights), sum(randoms_sum_weights)
    for icat, rw in enumerate(randoms_weights):
        if randoms_sum_weights[icat] != 0:
            factor = data_sum_weights[icat] / randoms_sum_weights[icat] * all_randoms_sum_weights / all_data_sum_weights
            rw[-1] *= factor
    return data_weights, randoms_weights

def concatenate_data_randoms(data_positions_weights, *randoms_positions_weights, weight_attrs=None, randoms_splits_size=None, concatenate=None, mpicomm=None):
    # data_positions_weights: list of (positions, weights) (for each region)
    # randoms_positions_weights: list of (positions, weights) (for each region)
    def _concatenate(array):
        if isinstance(array[0], (tuple, list)):
            array = [np.concatenate([arr[iarr] for arr in array], axis=0) if array[0][iarr] is not None else None for iarr in range(len(array[0]))]
        elif array is not None:
            array = np.concatenate(array)  # e.g. Z column
        return array
    data_positions, data_weights = tuple(_concatenate([pw[i] for pw in data_positions_weights]) for i in range(len(data_positions_weights[0]) - 1)), [pw[-1] for pw in data_positions_weights]
    if not randoms_positions_weights:
        data_weights = _concatenate(data_weights)
        return data_positions + (data_weights,)
    print('data proceed finish here',flush=True)
    list_randoms_positions = tuple([_concatenate([pw[i] for pw in rr]) for rr in randoms_positions_weights] for i in range(len(randoms_positions_weights[0][0]) - 1))
    list_randoms_weights = [[pw[-1] for pw in rr] for rr in randoms_positions_weights]
    print('random proceed finish here',flush=True)
    import pdb; pdb.set_trace()
    for iran, randoms_weights in enumerate(list_randoms_weights):
        list_randoms_weights[iran] = _concatenate(normalize_data_randoms_weights(data_weights, randoms_weights, weight_attrs=weight_attrs)[1])
    data_weights = _concatenate(data_weights)
    list_randoms_positions_weights = list_randoms_positions + (list_randoms_weights,)
    if concatenate:
        list_randoms_positions_weights = tuple(_concatenate(rr) for rr in list_randoms_positions_weights)
    return data_positions + (data_weights,), list_randoms_positions_weights


In [5]:
c = 299792 # speed of light in km/s
cosmo = AbacusSummit() 
tracer = 'LRG'
region = 'NGC'
nsplits = 10
zrange = Y3_ZRANGE[tracer][0]
basedir = f'/pscratch/sd/s/shengyu/galaxies/catalogs/DA2/Abacus_v4_1/altmtl0/kibo-v1/mock0/LSScats'
ran_mock_fns = [basedir+f'/{tracer}_{region}_{{}}_clustering.ran.fits'.format(i) for i in range(2)]
ran_mocks = vstack([Table(fitsio.read(v, columns=['RA', 'DEC', 'Z', 'WEIGHT'])) for v in ran_mock_fns])
randoms = get_positions_weights(np.array_split(ran_mocks, nsplits), tracer, region, zrange, weight_type = 'default', systematics_type = None)

data_mock_fn = basedir+f'/{tracer}_NGC_clustering.dat.fits'
data_mocks=Table(fitsio.read(data_mock_fn, columns=['RA', 'DEC', 'Z', 'WEIGHT']))
data = get_positions_weights(data_mocks, tracer, region, zrange, weight_type = 'default', systematics_type = None)

In [6]:
import numpy as np
from pycorr import mpi
c = 299792 # speed of light in km/s
cosmo = AbacusSummit() 
def _concatenate(array):
    if isinstance(array[0], (tuple, list)):
        array = [np.concatenate([arr[iarr] for arr in array], axis=0) if array[0][iarr] is not None else None for iarr in range(len(array[0]))]
    elif array is not None:
        array = np.concatenate(array)  # e.g. Z column
    return array
data_positions, data_weights = tuple(_concatenate([pw[i] for pw in data]) for i in range(len(data[0]) - 1)), [pw[-1] for pw in data]
print('data proceed finish here',flush=True)
# list_randoms_positions = tuple([_concatenate([pw[i] for pw in rr]) for rr in randoms] for i in range(len(randoms[0][0]) - 1))
# list_randoms_weights = [[pw[-1] for pw in rr] for rr in randoms]
# print('random proceed finish here',flush=True)

: 